# Recap on Pandas

This notebook recaps the core concepts on Pandas (data structures, aggregation operations, string methods, label and position indexing, groupby) before a deeper dive on time series data.

Authors: [Alexandre Gramfort](http://alexandre.gramfort.net)
         [Thomas Moreau](https://tommoral.github.io)

(based on Wes McKinney's book "Python for Data Analysis" O'REILLY,
and the pandas materials from Joris Van den Bossche and Stijn Van Hoey,
as well as the Python Data Science Handbook by Jake VanderPlas)

Licensed under [CC BY 4.0 Creative Commons](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
import pandas as pd  # THE standard way of importing pandas

Some imports

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-colorblind')

# Recap on Data structures

Pandas provides two fundamental data objects, for 1D (``Series``) and 2D data (``DataFrame``).

## Series

A Series is a basic holder for **one-dimensional labeled data**. It can be created much as a NumPy array is created:

In [ ]:
s = pd.Series([0.1, 0.2, 0.3, 0.4])
s

In [ ]:
type(s)

### Attributes of a Series: `index` and `values`

The series has a built-in concept of an **index**, which by default is the numbers *0* through *N - 1*

In [ ]:
s.index

You can access the underlying numpy array representation with the `.values` attribute:

In [ ]:
s.values

In [ ]:
type(s.values)

We can access series values via the index, just like for NumPy arrays:

In [ ]:
s[0]

Unlike the NumPy array, though, this index can be something other than integers:

In [ ]:
s2 = pd.Series(np.arange(4), index=['a', 'b', 'c', 'd'])
s2

In [ ]:
s2['c']

### Pandas Series versus dictionaries

In this way, a ``Series`` object can be thought of as similar to an ordered dictionary mapping one typed value to another typed value.

In fact, it's possible to construct a series directly from a Python dictionary:

In [ ]:
pop_dict = {'Germany': 81.3, 
            'Belgium': 11.3, 
            'France': 64.3, 
            'United Kingdom': 64.9, 
            'Netherlands': 16.9}
population = pd.Series(pop_dict)
population

We can index the populations like a dict as expected:

In [ ]:
population['France']

but with the power of numpy arrays:

In [ ]:
population * 1000

## DataFrames: Multi-dimensional Data

A DataFrame is a **tablular data structure** (multi-dimensional object to hold labeled data) comprised of rows and columns, akin to a spreadsheet, database table, or R's data.frame object. You can think of it as multiple Series object which share the same index.

<img src="img/schema-dataframe.svg" width=50%>

One of the most common ways of creating a dataframe is from a dictionary of arrays or lists.

Note that in the IPython notebook, the dataframe will display in a rich HTML view:

In [ ]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

In [ ]:
type(countries)

In [ ]:
type(countries['area'])

In [ ]:
countries['area']

### Attributes of the DataFrame

A DataFrame has besides a `index` attribute, also a `columns` attribute:

In [ ]:
countries.index

In [ ]:
countries.columns

To check the data types of the different columns:

In [ ]:
countries.dtypes

An overview of that information can be given with the `info()` method:

In [ ]:
countries.info()

Also a DataFrame has a `values` attribute, but attention: when you have heterogeneous data, all values will be upcasted:

In [ ]:
countries.values

To access a Series representing a column in the data, use typical indexing syntax:

In [ ]:
countries['area']

### Changing the DataFrame index

If we don't like what the index looks like, we can reset it and set one of our columns:

In [ ]:
countries = countries.set_index('country')
countries

Reversing this operation, is `reset_index`:

In [ ]:
countries.reset_index('country')

In [ ]:
countries

## Some useful methods on these data structures

Exploration of the Series and DataFrame is essential (check out what you're dealing with). 

In [ ]:
countries.head(4) # Top rows

In [ ]:
countries.tail(4) # Bottom rows

One useful method to use is the ``describe`` method, which computes summary statistics for each column:

In [ ]:
countries.describe(include='all')

**Sort**ing your data **by** a specific column is another important first-check:

In [ ]:
countries.sort_values(by='population')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Check the help of the <code>sort_values</code> function and find out how to sort from the largest values to the lowest values</li>
    </ul>
</div>

# Importing and exporting data

A wide range of input/output formats are natively supported by pandas:

* CSV, text (everyone can read but it makes big files and suffers from ambiguous dtypes)
* SQL database
* Excel (great if you want to store multiple dataframes in one file as sheets)
* HDF5 (very standard format for scientific data, has good compression support)
* json (ideal for web stuff)
* html (for notebook and dashboards)
* pickle (when you store arbitraty objects in the DataFrame)
* parquet (the best alternative to .csv or .csv.gz files!)
* ...

In [ ]:
countries.to_parquet('countries.parquet')

##  Basic operations on Series and DataFrames

As you play around with DataFrames, you'll notice that many operations which work on NumPy arrays will also work on dataframes.

In [ ]:
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

countries = pd.DataFrame({'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
                          'population': [11.3, 64.3, 81.3, 16.9, 64.9],
                          'area': [30510, 671308, 357050, 41526, 244820],
                          'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']})

In [ ]:
countries.head()

## Aggregations (reductions)

Pandas provides a large set of **summary** functions that operate on different kinds of pandas objects (DataFrames, Series, Index) and produce single value. When applied to a DataFrame, the result is returned as a pandas Series (one value for each column). 

The average population number:

In [ ]:
population.mean()

The minimum area:

In [ ]:
countries['area'].min()

For dataframes, often only the numeric columns are included in the result:

In [ ]:
countries.median(numeric_only=True)

In [ ]:
countries.mean(numeric_only=True)

In [ ]:
type(countries.mean(numeric_only=True))

In [ ]:
countries.mean(numeric_only=True).index

## Pandas: Indexing and selecting data

In [ ]:
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

Setting the index to the country names:

In [ ]:
countries = countries.set_index('country')
countries

# Selecting data

<div class="alert alert-warning">
<b>ATTENTION!</b>: <br><br>

One of pandas' basic features is the labeling of rows and columns, but this makes indexing also a bit more complex compared to numpy. <br><br> We now have to distuinguish between:

 <ul>
  <li>selection by **label**</li>
  <li>selection by **position**</li>
</ul>
</div>

## `data[]` provides some convenience shortcuts 

For a DataFrame, basic indexing selects the columns (cfr. the dictionaries of pure python)

Selecting a **single column**:

In [ ]:
countries['area'] # single []

or multiple **columns**:

In [ ]:
countries[['area', 'population']] # double [[]]

But, slicing accesses the **rows**:

In [ ]:
a = np.arange(1, 10)
a

In [ ]:
a[0:2:1]

In [ ]:
countries['France':'Netherlands']

In [ ]:
countries

In [ ]:
df = countries.reset_index()
df[0:2]

<div class="alert alert-danger">
<b>NOTE</b>:

 <ul>
  <li>Unlike slicing in numpy, the end label is **included**!</li>
</ul>
</div>

<div class="alert alert-info">
<b>REMEMBER</b>: <br>

So as a summary, `[]` provides the following convenience shortcuts:

<ul>
  <li>**Series**: selecting a **label**: `s[label]`</li>
  <li>**DataFrame**: selecting a single or multiple **columns**: `df['col']` or `df[['col1', 'col2']]`</li>
  <li>**DataFrame**: slicing the **rows**: `df['row_label1':'row_label2']` or `df[mask]`</li>
</ul>
</div>

## Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

Selecting a single element:

In [ ]:
countries

In [ ]:
countries.loc['Germany', 'area']

But the row or column indexer can also be a list, slice, boolean array (see next section), ..

In [ ]:
countries.loc['France':'Germany', ['area', 'population']]

---
Selecting by position with `iloc` works similar as **indexing numpy arrays**:

In [ ]:
countries.iloc[0:2,1:3]

The different indexing methods can also be used to assign data:

In [ ]:
countries2 = countries.copy()
countries2.loc['Belgium':'Germany', 'population'] = 10

In [ ]:
countries2

<div class="alert alert-info">
<b>REMEMBER</b>: <br>

Advanced indexing with **loc** and **ïloc**

 <ul>
  <li>**loc**: select by label: `df.loc[row_indexer, column_indexer]`</li>
  <li>**iloc**: select by position: `df.iloc[row_indexer, column_indexer]`</li>
</ul>
</div>

## Boolean indexing (filtering)

Often, you want to select rows based on a certain condition. This can be done with 'boolean indexing' (like a where clause in SQL) and comparable to numpy. 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [ ]:
countries['area'] > 100000

In [ ]:
countries[countries['area'] > 100000]

<div class="alert alert-success">
<b>EXERCISE</b>:

 <ul>
  <li>Select the capital and the population column of those countries where the density is larger than 300</li>
</ul>
</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

 <ul>
  <li>Select all countries whose population density is between 100 and 300 people/km²</li>
</ul>
</div>

## Efficient filtering with `query`

When you want to select rows based on more complex conditions, using _boolean indexing_ can be either inefficient or tedious:

In [ ]:
countries

In [ ]:
countries[(countries['area'] > 100000) & (countries['population'] > 50)]

This will create 2 boolean masks in memory and then index `countries` so it will scan the `DataFrame` 3 times.

Successive indexing will be more efficient (working on sub samples of the rows) but will require creating many temporary variable:

In [ ]:
countries_area = countries[countries['area'] > 100000]
countries_area[countries_area['population'] > 50]

In order to avoid this drawbacks, one can use the `query` method from a `DataFrame`:

In [ ]:
countries.query('area > 1e5 & population > 50')

With this method, a single efficient query will be performed, evaluating and selecting the rows in  single efficient pass.

The `query` method accept complicated queries using a mini-language:

- column names are used as variable in the evaluted query
- `@foo` corresponds to the `foo` variable in the current namespace.
- special pandas method can be accessed if using `engine=python` (see bellow).

In [ ]:
min_density = 1e2
countries.query(
    'population * 1e6 / area > @min_density'
    '& capital.str.contains("B")',
    engine='python'
)

# Some other essential methods: `isin` and `string` methods

In [ ]:
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

The `isin` method of Series is very useful to select rows that may contain certain values:

In [ ]:
s = countries['capital']

In [ ]:
s.isin(['Berlin', 'London'])

This can then be used to filter the dataframe with boolean indexing:

In [ ]:
countries[countries['capital'].isin(['Berlin', 'London'])]

Let's say we want to select all data for which the capital starts with a 'B'. In Python, when having a string, we could use the `startswith` method:

In [ ]:
ss = 'Berlin'
ss.upper()

In [ ]:
'Berlin'.startswith('B')

In [ ]:
countries['capital'].apply(lambda x: x.startswith('B'))

In pandas, these are available on a Series through the `str` namespace:

In [ ]:
countries['capital'].str.startswith('B')

For an overview of all string methods, see: http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Select all countries that have capital names with more than 7 characters</li>
</ul>
    
</div>

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Select all countries that have capital names that contain the character sequence 'am'</li>
</ul>
</div>

## Combining datasets: concat and merge

Combining data is essential functionality in a data analysis workflow. 

Data is distributed in multiple files, different information needs to be merged, new data is calculated, .. and needs to be added together. Pandas provides various facilities for easily combining together Series and DataFrame objects

The ``pd.concat`` function does all of the heavy lifting of combining data in different ways.

``pd.concat`` takes a list or dict of Series/DataFrame objects and concatenates them in a certain direction (`axis`) with some configurable handling of “what to do with the other axes”.


## Combining rows - ``pd.concat``

![](img/schema-concat0.svg)

In [ ]:
# series
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

# dataframe
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

Assume we have some similar data as in `countries`, but for a set of different countries:

In [ ]:
data = {'country': ['Nigeria', 'Rwanda', 'Egypt', 'Morocco', ],
        'population': [182.2, 11.3, 94.3, 34.4],
        'area': [923768, 26338 , 1010408, 710850],
        'capital': ['Abuja', 'Kigali', 'Cairo', 'Rabat']}
countries_africa = pd.DataFrame(data)
countries_africa 

We now want to combine the rows of both datasets:

In [ ]:
pd.concat([countries, countries_africa], sort=False)

If we don't want the index to be preserved:

In [ ]:
pd.concat([countries, countries_africa], ignore_index=True, sort=False)

We can also pass a dictionary of objects instead of a list of objects. Now the keys of the dictionary are preserved as an additional index level:

In [ ]:
pd.concat({'europe': countries, 'africa': countries_africa}, sort=False)

## Combining columns  - ``pd.concat`` with ``axis=1``

![](img/schema-concat1.svg)

Assume we have another DataFrame for the same countries, but with some additional statistics:

In [ ]:
data = {'country': ['Belgium', 'France', 'Netherlands'],
        'GDP': [496477, 2650823, 820726],
        'area': [8.0, 9.9, 5.7]}
country_economics = pd.DataFrame(data).set_index('country')
country_economics

In [ ]:
pd.concat([countries, country_economics], axis=1)

`pd.concat` matches the different objects based on the index:

In [ ]:
countries2 = countries.set_index('country')

In [ ]:
countries2

In [ ]:
country_economics

In [ ]:
pd.concat([countries2, country_economics], axis="columns", sort=False)

# Joining data with `pd.merge`

Using `pd.concat` above, we combined datasets that had the same columns or the same index values. But, another typical case if where you want to add information of second dataframe to a first one based on one of the columns. That can be done with [`pd.merge`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html).

Let's look again at the titanic passenger data, but taking a small subset of it to make the example easier to grasp:

In [ ]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom', 'Australia'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9, 24.4],
        'area': [30510, 671308, 357050, 41526, 244820, 7692024],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London', 'Canberra'],
        'hemisphere': ['North', 'North', 'North', 'North', 'North', 'South']}
countries = pd.DataFrame(data)
countries

In [ ]:
data = {'city': ['Paris', 'Lyon', 'Lille', 'Berlin', 'Munich', 'Sydney'],
        'country': ['France', 'France', 'France', 'Germany', 'Germany', 'Australia']}
cities = pd.DataFrame(data)
cities

Let's say we want to know in what hemisphere is each city

In [ ]:
cities

In [ ]:
pd.merge(cities, countries, on='country', how='left')

In [ ]:
pd.merge(cities, countries[['country', 'hemisphere']],
         on='country', how='left')

In this case we use `how='left` (a "left join") because we wanted to keep the original rows of `cities` and only add matching values from `countries` to it. Other options are 'inner', 'outer' and 'right' (see the [docs](http://pandas.pydata.org/pandas-docs/stable/merging.html#brief-primer-on-merge-methods-relational-algebra) for more on this).

## "Group by" operations

### Some 'theory': the groupby operation (split-apply-combine)

In [ ]:
df = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                   'data': [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

### Aggregating functions

When analyzing data, you often calculate summary statistics (aggregations like the mean, max, ...). As we have seen before, we can easily calculate such a statistic for a Series or column using one of the many available methods. For example:

In [ ]:
np.sum(df['data'].values)

However, in many cases your data has certain groups in it, and in that case, you may want to calculate this statistic for each of the groups.

For example, in the above dataframe `df`, there is a column 'key' which has three possible values: 'A', 'B' and 'C'. When we want to calculate the sum for each of those groups, we could do the following:

In [ ]:
df[df['key'] == "A"]['data'].sum()

In [ ]:
print('A', df[df['key'] == "A"]['data'].sum())
print('B', df[df['key'] == "B"]['data'].sum())
print('C', df[df['key'] == "C"]['data'].sum())

This becomes very verbose when having multiple groups. You could make the above a bit easier by looping over the different values, but still, it is not very convenient to work with.

What we did above, applying a function on different groups, is a "groupby operation", and pandas provides some convenient functionality for this.

### Groupby: applying functions per group

The "group by" concept: we want to **apply the same function on subsets of your dataframe, based on some key to split the dataframe in subsets**

This operation is also referred to as the "split-apply-combine" operation, involving the following steps:

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

<img src="img/splitApplyCombine.png">

Similar to SQL `GROUP BY`

Instead of doing the manual filtering as above


    df[df['key'] == "A"].sum()
    df[df['key'] == "B"].sum()
    ...

pandas provides the `groupby` method to do exactly this:

In [ ]:
df.groupby('key').sum()

In [ ]:
df.groupby('key').aggregate(np.sum)  # 'sum'

And many more methods are available. 

In [ ]:
df.groupby('key')['data'].sum()

# Some more theory

## Specifying the grouper

In the previous example and exercises, we always grouped by a single column by passing its name. But, a column name is not the only value you can pass as the grouper in `df.groupby(grouper)`. Other possibilities for `grouper` are:

- a list of strings (to group by multiple columns)
- a Series (similar to a string indicating a column in df) or array
- function (to be applied on the index)
- dict : groups by values
- levels=[], names of levels in a MultiIndex

In [ ]:
df.groupby(df['data'] < 18).mean()

## The size of groups - value counts

Oftentimes you want to know how many elements there are in a certain group (or in other words: the number of occurences of the different values from a column).

To get the size of the groups, we can use `size`:

In [ ]:
df.groupby('key').size()

Another way to obtain such counts, is to use the Series `value_counts` method:

In [ ]:
df['key'].value_counts()